In [1]:
import pandas as pd
import os
import requests
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
datasets_path = "../datasets"

movie_filename = "top_movies.csv"
tv_filename = "top_tv.csv"

actors_filename = "actors.csv"
directors_filename = "directors.csv"

In [4]:
movies_df = pd.read_csv(os.path.join(datasets_path, movie_filename))
print(movies_df.info())

C:\Users\adars\AppData\Local\Temp\ipykernel_24172\1385942988.py:1: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(os.path.join(datasets_path, movie_filename))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110915 entries, 0 to 110914
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    110915 non-null  int64  
 1   title                 110915 non-null  object 
 2   vote_average          110915 non-null  float64
 3   vote_count            110915 non-null  int64  
 4   status                110915 non-null  object 
 5   release_date          110833 non-null  object 
 6   revenue               110915 non-null  int64  
 7   runtime               110915 non-null  int64  
 8   adult                 110915 non-null  bool   
 9   backdrop_path         90368 non-null   object 
 10  budget                110915 non-null  int64  
 11  homepage              20914 non-null   object 
 12  imdb_id               109599 non-null  object 
 13  original_language     110915 non-null  object 
 14  original_title        110915 non-null  object 
 15  

Some movie attributes we need for content based filtering: 

vote_count: how popular is the movie  
release_date: ...  
runtime: ...  
adult: ...  
original_language: ...  
genres: ...  
keywords: ...  

we will also need  
cast: people may prefer movies with some actors  
director: ...

The movies dataset has many attributes that we need for our purpose but some we dont have.  
We dont have a column for cast and the director which are necessary alongside genre and others for the content based filtering. 

To get data for these columns, we will use the TMDB API.  
We can pass the movie id to the API and get data for cast and director, and store that data in the respective columns.

In [33]:
tv_df = pd.read_csv(os.path.join(datasets_path, tv_filename))
print(tv_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16513 entries, 0 to 16512
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    16513 non-null  int64  
 1   name                  16513 non-null  object 
 2   number_of_seasons     16513 non-null  int64  
 3   number_of_episodes    16513 non-null  int64  
 4   original_language     16513 non-null  object 
 5   vote_count            16513 non-null  int64  
 6   vote_average          16513 non-null  float64
 7   overview              15708 non-null  object 
 8   adult                 16513 non-null  bool   
 9   backdrop_path         15597 non-null  object 
 10  first_air_date        16421 non-null  object 
 11  last_air_date         16439 non-null  object 
 12  homepage              11959 non-null  object 
 13  in_production         16513 non-null  bool   
 14  original_name         16513 non-null  object 
 15  popularity         

Some tv attributes we need for content based filtering:

number_of_season: ...  
number_of_episodes: ...  
original_language: ...  
vote_count: how popular is the movie  
adult: ...  
first_air_date: ...   
type: ...  
genres: ...  
created_by: ...

For TV dataset, we dont have cast, but it wont be much necessary because its not as important in TV compared to movies where actors play in many movies many of which are also popular. Besides, we already have created_by attribute which is enough for now. 

So now, we need to use the TMDB API for getting data of cast and director for our movie dataset and add those data in new columns.

We will create one new dataset, actors.csv with various details of the actors.

We will create two new columns in movies dataset:  

1. cast  
here we will store a list of 5 objects, each object for one actor. the object will have attributes: name, id, role (role played in that movie)  

2. director  
here we will store a single object, with attributes: name, id

In [34]:
actors_df = pd.read_csv(os.path.join(datasets_path, actors_filename))
directors_df = pd.read_csv(os.path.join(datasets_path, directors_filename))

In [42]:
# To find out the movie index from which we need to update the actor and director column
new_starting_point = movies_df[movies_df["cast"].isna()].index[0]
print(new_starting_point)

5000


In [38]:
TMDB_API_TOKEN = os.environ.get("TMDB_API_TOKEN")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {TMDB_API_TOKEN}"
}

for index, row in movies_df[4500:5000].iterrows():
    url = f"https://api.themoviedb.org/3/movie/{row['id']}/credits?language=en-US"
    response = requests.get(url, headers=headers).json()

    cast_list = response["cast"]  # cast array
    crew_list = response["crew"]  # crew array

    top_cast = cast_list[:5]  # top 5 cast

    # find director from crew array
    director = next((person for person in crew_list if person["job"] == "Director"), None)

    # add actors to actors_df if not already present
    for actor in top_cast:
        if not (actors_df["id"] == actor["id"]).any():
            actors_df.loc[len(actors_df)] = [
                actor.get("adult"),
                actor.get("gender"),
                actor.get("id"),
                actor.get("known_for_department"),
                actor.get("name"),
                actor.get("original_name"),
                actor.get("popularity"),
                actor.get("profile_path")
            ]

    # add director to directors_df if not already present
    if director and not (directors_df["id"] == director["id"]).any():
        directors_df.loc[len(directors_df)] = [
            director.get("adult"),
            director.get("gender"),
            director.get("id"),
            director.get("known_for_department"),
            director.get("name"),
            director.get("original_name"),
            director.get("popularity"),
            director.get("profile_path")
        ]

    # keep only necessary actor attributes for the movie table
    top_cast_filtered = [
        {
            "cast_id": actor.get("cast_id"),
            "character": actor.get("character"),
            "order": actor.get("order"),
            "name": actor.get("name"),
            "id": actor.get("id")
        }
        for actor in top_cast
    ]
    movies_df.at[index, "cast"] = top_cast_filtered

    # keep only necessary director attributes for the movie table
    if director:
        director_filtered = {
            "id": director.get("id"),
            "name": director.get("name")
        }
        movies_df.at[index, "director"] = director_filtered


In [39]:
# saving the updated movies table, actors table, directors table
movies_df.to_csv(os.path.join(datasets_path, movie_filename), index=False)
actors_df.to_csv(os.path.join(datasets_path, actors_filename), index=False)
directors_df.to_csv(os.path.join(datasets_path, directors_filename), index=False)

In [41]:
movies_df = pd.read_csv(os.path.join(datasets_path, movie_filename))
actors_df = pd.read_csv(os.path.join(datasets_path, actors_filename))
directors_df = pd.read_csv(os.path.join(datasets_path, directors_filename))

print(len(actors_df))
print(len(directors_df))
print(movies_df.loc[4999])

9642
2060
id                                                                 790142
title                                                 The Royal Treatment
vote_average                                                        6.708
vote_count                                                            714
status                                                           Released
release_date                                                   2022-01-20
revenue                                                                 0
runtime                                                                96
adult                                                               False
backdrop_path                            /on1p1AdcfxmlyHhVXoycWt75D8H.jpg
budget                                                                  0
homepage                        https://www.netflix.com/br/title/81228331
imdb_id                                                        tt13989030
original_language           

C:\Users\adars\AppData\Local\Temp\ipykernel_24172\1354414482.py:1: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(os.path.join(datasets_path, movie_filename))


This completes the code for using the TMDB API to add  
1. actors table  
2. directors table  
3. actors and directors basic information in the movies table  

Currently, the last movie's index whose cast and director column has been updated is  
1499